# Loading data

In [1]:
import keras

c:\users\bikas\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Preprocessing data

In [15]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.

data_path = 'data\\eng-fra.txt'

input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
    
print("Total number of translation pair : ", len(lines))

Total number of translation pair :  149862


In [16]:
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text = line.split('\t')
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [17]:
print("input characters: ", input_characters)
print("target characters: ", target_characters)

# sample translation
for i in range(5):
  print(input_texts[i]+ "    --->    "+target_texts[i])

input characters:  {'&', 'R', ':', 'h', '’', '!', 'l', 'F', 'g', 'u', 'B', 'C', 'K', 'b', '9', '3', 'n', 'D', 'p', 'e', 'k', '$', '1', 'd', 'q', 'P', '4', 'T', 'c', 'E', 'J', 's', '.', 'x', '0', 'S', 'r', 'A', '?', 'M', 'i', 'L', 'N', "'", 'j', 'U', '5', 'a', 'I', 'v', 't', 'f', 'z', 'y', '6', '-', '2', 'H', 'w', '7', 'G', 'O', '8', ',', 'Y', 'V', 'Q', 'm', 'W', 'o', ' '}
target characters:  {'\n', 'h', '!', 'ï', 'B', 'C', 'd', 'c', 'x', '0', 'A', '?', "'", '6', 'Q', '8', '‘', '1', '’', 'F', '3', 'ê', 'D', 'e', '(', 'M', 'L', '5', 'j', 'z', '\u2009', 'è', 'H', 'G', 'ë', 'm', '.', ' ', 'à', ':', 'ô', 'b', 'K', ')', 'ç', 'k', 'œ', 'q', 'P', 'Ê', 'É', 's', 'J', 'î', 'N', 'U', 'v', 't', 'ù', 'w', 'O', '\u202f', 'Y', 'o', 'À', '&', 'R', '\t', 'l', '»', 'g', 'u', 'é', '9', 'n', 'p', '$', '\xa0', 'T', 'E', '«', 'Ç', 'S', 'r', 'i', 'a', 'I', 'f', 'â', 'y', '-', 'û', ',', 'V'}
Go.    --->    	Va !

Run!    --->    	Cours !

Run!    --->    	Courez !

Wow!    --->    	Ça alors !

Fire!    --->  

In [18]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])


print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 10000
Number of unique input tokens: 71
Number of unique output tokens: 94
Max sequence length for inputs: 16
Max sequence length for outputs: 59


### Assign token index

In [19]:
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])

target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

### Prepare one-hot encoder data representation : Shape (batch, sentence_len, encode_dim)

In [20]:
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

print("encoder input data shape: ", encoder_input_data.shape)
print("decoder input data shape: ", decoder_input_data.shape)
print("decoder target data shape: ", decoder_target_data.shape)

encoder input data shape:  (10000, 16, 71)
decoder input data shape:  (10000, 59, 94)
decoder target data shape:  (10000, 59, 94)


### decoder input :   HELLO    --->   decoder target:  ELLO

In [21]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.


# Build the Sequence to Sequence model

In [22]:
# Define input sequence
encoder_inputs = Input(shape=(None, num_encoder_tokens))
# LSTM as encoder
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# we only need 'encoder_outputs'
encoder_states = [state_h, state_c]


# set up the decoder, using `encoder_states` as initial state
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# we set up decoder to return full output sequences, and to return internal
# states as well. we don't use the return states in training model, but
# we will use them in referance
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)

decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# define a model that will turn 'encoder_input_data' & 'decoder_input_data'
# into 'decoder_target_data'
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [23]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, None, 71)     0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, None, 94)     0                                            
__________________________________________________________________________________________________
lstm_3 (LSTM)                   [(None, 256), (None, 335872      input_3[0][0]                    
__________________________________________________________________________________________________
lstm_4 (LSTM)                   [(None, None, 256),  359424      input_4[0][0]                    
                                                                 lstm_3[0][1]                     
          

In [24]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2
         )


Train on 8000 samples, validate on 2000 samples
Epoch 1/100
8000/8000 [==============================] - 11s 1ms/step - loss: 0.9278 - val_loss: 0.9819
Epoch 2/100
8000/8000 [==============================] - 10s 1ms/step - loss: 0.7473 - val_loss: 0.8046
Epoch 3/100
8000/8000 [==============================] - 10s 1ms/step - loss: 0.6305 - val_loss: 0.7197
Epoch 4/100
8000/8000 [==============================] - 9s 1ms/step - loss: 0.5715 - val_loss: 0.6783
Epoch 5/100
8000/8000 [==============================] - 9s 1ms/step - loss: 0.5293 - val_loss: 0.6369
Epoch 6/100
8000/8000 [==============================] - 9s 1ms/step - loss: 0.4967 - val_loss: 0.5976
Epoch 7/100
8000/8000 [==============================] - 9s 1ms/step - loss: 0.4705 - val_loss: 0.5804
Epoch 8/100
8000/8000 [==============================] - 9s 1ms/step - loss: 0.4475 - val_loss: 0.5601
Epoch 9/100
8000/8000 [==============================] - 9s 1ms/step - loss: 0.4284 - val_loss: 0.5479
Epoch 10/100
8000/8000

8000/8000 [==============================] - 9s 1ms/step - loss: 0.0813 - val_loss: 0.6923
Epoch 80/100
8000/8000 [==============================] - 11s 1ms/step - loss: 0.0803 - val_loss: 0.6960
Epoch 81/100
8000/8000 [==============================] - 11s 1ms/step - loss: 0.0788 - val_loss: 0.7009
Epoch 82/100
8000/8000 [==============================] - 11s 1ms/step - loss: 0.0776 - val_loss: 0.7064
Epoch 83/100
8000/8000 [==============================] - 11s 1ms/step - loss: 0.0769 - val_loss: 0.7114
Epoch 84/100
8000/8000 [==============================] - 11s 1ms/step - loss: 0.0757 - val_loss: 0.7123
Epoch 85/100
8000/8000 [==============================] - 10s 1ms/step - loss: 0.0747 - val_loss: 0.7151
Epoch 86/100
8000/8000 [==============================] - 11s 1ms/step - loss: 0.0739 - val_loss: 0.7133
Epoch 87/100
8000/8000 [==============================] - 11s 1ms/step - loss: 0.0721 - val_loss: 0.7186
Epoch 88/100
8000/8000 [==============================] - 11s 1ms/ste

In [25]:
model.save('saved_models\\s2s.h5')

c:\users\bikas\anaconda3\lib\site-packages\keras\engine\topology.py:2368: UserWarning: Layer lstm_4 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_3/while/Exit_2:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'lstm_3/while/Exit_3:0' shape=(?, 256) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  str(node.arguments) + '. They will not be included '


# Inferance with this model

In [26]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_state_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_state_inputs)

decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)

decoder_model = Model(
    [decoder_inputs]+decoder_state_inputs,
    [decoder_outputs]+decoder_states
)

### One-hot output to char

In [27]:
# Reverse-lookup token index to decode sequences back to something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

### Decode sequence

In [28]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [29]:
for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: Go.
Decoded sentence: Va !

-
Input sentence: Run!
Decoded sentence: Courez !

-
Input sentence: Run!
Decoded sentence: Courez !

-
Input sentence: Wow!
Decoded sentence: Ça alors !

-
Input sentence: Fire!
Decoded sentence: Au feu !

-
Input sentence: Help!
Decoded sentence: À l'aide !

-
Input sentence: Jump.
Decoded sentence: Saute.

-
Input sentence: Stop!
Decoded sentence: Ça suffit !

-
Input sentence: Stop!
Decoded sentence: Ça suffit !

-
Input sentence: Stop!
Decoded sentence: Ça suffit !

-
Input sentence: Wait!
Decoded sentence: Attendez !

-
Input sentence: Wait!
Decoded sentence: Attendez !

-
Input sentence: Go on.
Decoded sentence: Continuez.

-
Input sentence: Go on.
Decoded sentence: Continuez.

-
Input sentence: Go on.
Decoded sentence: Continuez.

-
Input sentence: I see.
Decoded sentence: Je vois un coupon.

-
Input sentence: I try.
Decoded sentence: J'essaye.

-
Input sentence: I won!
Decoded sentence: J'ai été discupé.

-
Input sentence: I won!
D